In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 7 * 7, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 128 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the DeepLIFT function
def deeplift(model, x, target_class):
    # Get the gradients of the output with respect to the input
    grads = torch.autograd.grad(model(x), x, retain_graph=True)[0]

    # Compute the relevance scores
    R = torch.zeros_like(grads)
    for i in range(grads.shape[0]):
        for j in range(grads.shape[1]):
            for k in range(grads.shape[2]):
                for l in range(grads.shape[3]):
                    R[i, j, k, l] = torch.sum(grads[i, j, k, l] * torch.abs(torch.autograd.grad(grads[i, j, k, l], x)[0]))

    # Normalize the relevance scores
    R = R

In [6]:
model=MyModel()
input_tensor = torch.tensor([[0.2, 0.4, 0.6, 0.8]], requires_grad=True)
target_class = [1,1,0,0,1]
baseline_tensor = torch.zeros_like(input_tensor, requires_grad=True)
attribution_scores = deeplift(model, input_tensor, target_class )

# The 'attribution_scores' tensor now contains the DeepLIFT attribution scores for each input feature
print(attribution_scores)


RuntimeError: ignored